<a href="https://colab.research.google.com/github/honicky/character-extraction/blob/main/Label_Stories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai datasets

In [ ]:
from datasets import load_dataset

honicky_dataset = load_dataset('honicky/short_childrens_stories_with_labeled_character_names')
loubnabnl_dataset = load_dataset('honicky/stories_oh_children_with_character_names')



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from openai import OpenAI
from google.colab import userdata
import json

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

character_prompt_template = """Please analyze the following story and identify the main characters.
Output the result in JSON format with a "characters" array containing the names of the main characters

<story>
{story}
</story>
"""

def extract_characters_using_openai(story, model="gpt-3.5-turbo"):
  response = client.chat.completions.create(
    model=model,
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": character_prompt_template.format(story=story) }
    ],
    max_tokens=200,
  )
  return json.loads(response.choices[0].message.content).get('characters', ['ERROR'])


In [ ]:
chunk_size = 100
characters = []

for chunk_start in range(0, len(loubnabnl_dataset['train']), chunk_size):

  characters += [
    extract_characters_using_openai(story)
    for story in loubnabnl_dataset['train']['completion'][chunk_start:chunk_start+chunk_size]
  ]
  print('.', end="", flush=True)

..................................................

In [ ]:
characters[-5:]

[['Fiona Fox', 'Benny Bunny'],
 ['Indira'],
 ['Little Lily', "Lily's mother"],
 ['Max', 'Charlie'],
 ['Lucy', 'Max']]

In [ ]:
import pandas as pd

loubnabnl_pd = pd.DataFrame(loubnabnl_dataset)
loubnabnl_pd["characters"] = pd.Series([
    ','.join(character_list)
    for character_list in characters
])

loubnabnl_pd.to_parquet('/content/drive/MyDrive/Learning/story-time/loubnabnl_stories.parquet')

# Upload manually

Sorry folks, I uploaded this dataset to huggingface manually: [honicky/stories_oh_children_with_character_names](https://huggingface.co/datasets/honicky/stories_oh_children_with_character_names)